In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
input_path = "test7_3.csv"
output_path = "testout7_3.csv"

In [4]:
df = pd.read_csv(input_path)
y = pd.to_numeric(df["y"], errors="coerce").dropna()
X = df[["x1", "x2", "x3"]].apply(pd.to_numeric, errors="coerce")
X = sm.add_constant(X, has_constant="add")

In [7]:
class TRegression(GenericLikelihoodModel):
    def loglike(self, params):
        k = self.exog.shape[1]
        beta = params[:k]
        log_sigma = params[k]
        log_df = params[k + 1]

        sigma = np.exp(log_sigma)
        df = np.exp(log_df) + 2
        resid = self.endog - np.dot(self.exog, beta)
        ll = stats.t.logpdf(resid, df, loc=0, scale=sigma)
        return np.sum(ll)

In [9]:
ols_res = sm.OLS(y, X).fit()
beta0 = ols_res.params
sigma0 = np.sqrt(ols_res.scale)
df0 = 10.0
start_params = np.r_[beta0, np.log(sigma0), np.log(df0)]

model = TRegression(y, X)
fit_res = model.fit(start_params=start_params, disp=False)

k = X.shape[1]
beta = fit_res.params[:k]
sigma = np.exp(fit_res.params[k])
nu = np.exp(fit_res.params[k + 1]) + 2
mu = 0.0

columns = ["mu", "sigma", "nu"] + ["Alpha"] + [f"B{i}" for i in range(1, k)]
values = [mu, sigma, nu] + beta.tolist()

result_df = pd.DataFrame([values], columns=columns)
result_df.to_csv(output_path, index=False)